In [97]:
import uuid
from faker import Faker
from datetime import datetime, timedelta
import pyodbc
import random
import csv

In [38]:
server = 'XMG' 
database = 'supermarket'

In [39]:
conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
cursor = conn.cursor()


In [40]:
fake = Faker()

In [41]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Account' and xtype='U')
    CREATE TABLE Account (
        AccountID UNIQUEIDENTIFIER PRIMARY KEY,
        Email NVARCHAR(255) UNIQUE,
        DateRegistered DATETIME
    )
''')
conn.commit()

In [42]:
def generate_mock_accounts(num_accounts):
    accounts = []
    generated_emails = set()
    while len(accounts) < num_accounts:
        account_id = str(uuid.uuid4())
        email = Faker().email()
        if email not in generated_emails:
            date_registered = Faker().date_between(start_date='-5y', end_date='today').strftime('%Y-%m-%d %H:%M:%S')
            accounts.append((account_id, email, date_registered))
            generated_emails.add(email)
    return accounts

In [43]:
mock_accounts = generate_mock_accounts(100)

In [44]:
cursor.executemany('INSERT INTO Account (AccountID, Email, DateRegistered) VALUES (?, ?, ?)', mock_accounts)
conn.commit()

In [50]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Town' and xtype='U')
    CREATE TABLE Town (
        TownID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        Name NVARCHAR(255) NOT NULL UNIQUE
    );
''')
conn.commit()

In [52]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Profile' and xtype='U')
    CREATE TABLE Profile (
        ProfileID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        DisplayName NVARCHAR(255) NOT NULL,
        House NVARCHAR(255),
        Locality NVARCHAR(255),
        TownID UNIQUEIDENTIFIER NOT NULL,
        AccountID UNIQUEIDENTIFIER NOT NULL UNIQUE,
        FOREIGN KEY (AccountID) REFERENCES Account(AccountID),
        FOREIGN KEY (TownID) REFERENCES Town(TownID)
    );  
''')
conn.commit()

In [47]:
cursor.execute("SELECT TOP 60 AccountID FROM Account ORDER BY NEWID()")
accounts = cursor.fetchall()
account_ids = [account[0] for account in accounts]

In [49]:
fake = Faker('en_GB')

In [53]:
towns = {fake.city() for _ in range(20)}  # Generate a set of unique towns
for town in towns:
    cursor.execute("IF NOT EXISTS (SELECT * FROM Town WHERE Name = ?) INSERT INTO Town (Name) VALUES (?)", town, town)
conn.commit()

In [55]:
def generate_profiles(account_ids):
    profiles = []
    for account_id in account_ids:
        display_name = fake.name()
        house = fake.building_number()
        locality = fake.street_name()
        town = random.choice(list(towns)) 

        cursor.execute("SELECT TownID FROM Town WHERE Name = ?", town)
        town_id = cursor.fetchone()[0]

        profiles.append((display_name, house, locality, town_id, account_id))

    return profiles

In [58]:
profiles = generate_profiles(account_ids)
cursor.executemany("INSERT INTO Profile (DisplayName, House, Locality, TownID, AccountID) VALUES (?, ?, ?, ?, ?)", profiles)
conn.commit()


In [66]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='ProductCompany' and xtype='U')
    CREATE TABLE ProductCompany (
        CompanyID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        CompanyName NVARCHAR(255) NOT NULL UNIQUE
    );
''')
conn.commit()

In [67]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Category' and xtype='U')
    CREATE TABLE Category (
        CategoryID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        CategoryName NVARCHAR(255) NOT NULL UNIQUE
    );
''')
conn.commit()

In [91]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Products' and xtype='U')
    CREATE TABLE Products (
        ProductID UNIQUEIDENTIFIER PRIMARY KEY,
        Barcode NVARCHAR(255) UNIQUE,
        ProductName NVARCHAR(255) NOT NULL,
        CompanyID UNIQUEIDENTIFIER NOT NULL,
        CategoryID UNIQUEIDENTIFIER NOT NULL,
        ProductSize INT NOT NULL,
        ProductPrice FLOAT NOT NULL,
        FOREIGN KEY (CompanyID) REFERENCES ProductCompany(CompanyID),
        FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID),
    );
''')
conn.commit()

In [92]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='ProductRating' and xtype='U')
    CREATE TABLE ProductRating (
        RatingID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        ProductID UNIQUEIDENTIFIER NOT NULL,
        ProductRate FLOAT,
        OneStar INT,
        TwoStar INT,
        ThreeStar INT,
        FourStar INT,
        FiveStar INT,
        ReviewsNumber INT,
        ProductQuality NVARCHAR(50),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    );
''')
conn.commit()

In [93]:
def get_or_create_id(table, id, column, value):
    cursor.execute(f"SELECT {id} FROM {table} WHERE {column} = ?", value)
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute(f"INSERT INTO {table} ({column}) VALUES (?)", value)
        conn.commit()
        return cursor.execute(f"SELECT {id} FROM {table} WHERE {column} = ?", value).fetchone()[0]

In [94]:
with open('products.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if (row['product_price']=='not availabe'):
            continue
        # Normalize data
        company_id = get_or_create_id('ProductCompany', 'CompanyID', 'CompanyName', row['product_company'].strip())
        category_id = get_or_create_id('Category', 'CategoryID', 'CategoryName', row['category'].strip())

        product_id = str(uuid.uuid4())
        
        cursor.execute(
            "INSERT INTO Products (ProductID, Barcode, ProductName, CompanyID, CategoryID, ProductSize, ProductPrice) VALUES (?, ?, ?, ?, ?, ?, ?)",
                product_id, fake.ean13(), row['product_name'].strip(), company_id, category_id, int(row['product_size'].split()[0].replace('g','')), row['product_price']
            )
        conn.commit()

        cursor.execute(
            "INSERT INTO ProductRating (ProductID, ProductRate, OneStar, TwoStar, ThreeStar, FourStar, FiveStar, ReviewsNumber, ProductQuality) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
            product_id, row['product_rate'], row['one_star'], row['two_star'], row['three_star'], row['four_star'], row['five_star'], row['reviews_number'], row['product_quality']
        )
        conn.commit()




In [95]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Orders' and xtype='U')
    CREATE TABLE Orders (
        OrderID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        AccountID UNIQUEIDENTIFIER NOT NULL,
        OrderDate DATETIME NOT NULL,
        FOREIGN KEY (AccountID) REFERENCES Account(AccountID)
    );
''')
conn.commit()

In [98]:
cursor.execute("SELECT TOP 90 AccountID, DateRegistered FROM Account ORDER BY NEWID()")
accounts = cursor.fetchall()

# Function to generate orders
def generate_orders(account_id, date_registered):
    num_orders = random.randint(1, 30)  # Random number of orders (1 to 30)
    orders = []
    for _ in range(num_orders):
        # Order date must be after date registered and not more than 5 years ago
        start_date = max(date_registered, datetime.now() - timedelta(days=5*365))
        order_date = fake.date_time_between_dates(datetime_start=start_date, datetime_end=datetime.now())
        orders.append((account_id, order_date))
    return orders

# Generate and insert orders for each account
for account_id, date_registered in accounts:
    orders = generate_orders(account_id, date_registered)
    cursor.executemany("INSERT INTO Orders (AccountID, OrderDate) VALUES (?, ?)", orders)
    conn.commit()

In [102]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='OrderItem' and xtype='U')
    CREATE TABLE OrderItem (
        OrderID UNIQUEIDENTIFIER NOT NULL,
        ProductID UNIQUEIDENTIFIER NOT NULL,
        Quantity INT NOT NULL,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID),
        PRIMARY KEY(OrderId, ProductId)
    );
''')
conn.commit()

In [103]:
# Fetch all OrderIDs
cursor.execute("SELECT OrderID FROM Orders")
orders = [row[0] for row in cursor.fetchall()]

# Fetch all ProductIDs
cursor.execute("SELECT ProductID FROM Products")
products = [row[0] for row in cursor.fetchall()]

# Function to generate order items for an order
def generate_order_items(order_id, products):
    num_products = random.randint(1, 30)  # Random number of unique products
    selected_products = random.sample(products, num_products)
    order_items = [(order_id, product_id, random.randint(1, 5)) for product_id in selected_products]  # Random quantity for each product
    return order_items

# Generate and insert order items for each order
for order_id in orders:
    order_items = generate_order_items(order_id, products)
    cursor.executemany("INSERT INTO OrderItem (OrderID, ProductID, Quantity) VALUES (?, ?, ?)", order_items)
    conn.commit()
